## installation

In [3]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:
!pip install langchain_openai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [11]:
!pip install chromadb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 526 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 107 kB 51.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 49.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 36.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 24.4 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 43.4 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 59 kB 56.1 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 42.2 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 24.0 MB/s eta 0:00:01
     |████████████████████████████████| 283 kB 43.2 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 6.6 MB/s  eta 0:00:01
     |████████████████████████████████| 67 kB 46

In [13]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"



import shutil
import openai
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHROMA_PATH = "chroma"
DATA_PATH = "/workspace/test/07_chatbot/complete/data/book"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

def load_documents():
    loader = PyPDFLoader("data/book/PA00T6HT.pdf")
    pages = loader.load_and_split()
    return pages

def split_text(documents):
    text_splitter = RecursiveCharacterTextSplitter()
    return text_splitter.split_documents(documents)

def save_to_chroma(chunks):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

def run_query(query_text, embedding_function, db, model, prompt_template):
    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results in the database for: {query_text}")
        print("Using OpenAI to get a general response.")

        # OpenAI fallback
        openai.api_key = os.getenv("OPENAI_API_KEY")
        messages = [
            {"role": "system", "content": "You are a helpful assistant. Help the user with their questions."},
            {"role": "user", "content": query_text}
        ]
        response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens = 15
    )
        assistant_response = response.choices[0].message.content
        print(f"OpenAI Response: {assistant_response}")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(prompt)

    # Generate response using the model
    response_text = model.predict(prompt)

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)

# Function to run the chat bot continuously
def run_chat_bot():
    print("Welcome to the Kindergarten Q&A Bot!")
    print("Ask me any question about the kindergarten room. Type 'quit' to exit.")

    # Initialize components (simulated for example)
    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
    model = ChatOpenAI()
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

    while True:
        user_input = input("You: ").strip().lower()

        if user_input == "quit":
            print("Goodbye! Have a great day!")
            break

        # Call run_query function to handle the query and response
        run_query(user_input, embedding_function, db, model, prompt_template)
        print()  # Print an empty line for readability

# Run the chat bot
generate_data_store()
run_chat_bot()


/opt/conda/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


Saved 503 chunks to chroma.
Welcome to the Kindergarten Q&A Bot!
Ask me any question about the kindergarten room. Type 'quit' to exit.


You:  안녕하세요


Unable to find matching results in the database for: 안녕하세요
Using OpenAI to get a general response.
OpenAI Response: 안녕하세요! 어떻게 도와드릴까요?



You:  tell me about kid's Physical and Motor Development


Human: 
Answer the question based only on the following context:

UNDERSTANDING KINDER GARTEN TEACHING AND LEARNING – DAY 1  6 PHYSICAL DEVELOPMENT 
4 YEARS OLD  5 YEARS OLD  6-7 YEARS OLD  
Jump own height and land upright; 
jump over objects  
Hop, skip, stand on 1 foot  
Throw large ball, kick accurately  
Walk on a straight line 
Turn somersaults  
Alternate feet in going up/down stairs 
Have sureness and control in 
finger activities 
Hold paint brush in adult manner  
Draw a stick figure  
Lace shoes  Tremendous physical drive  
Catch ball from three  feet 
Skip using alternate feet  
Enjoy running, jumping, doing 
stunts  
Balance on a balance beam 
Jump rope  
Graceful, rhythmic dancer  
Draw recognizable person  
Dress self  
Color within lines  
Cut on a line with scissors  Boisterous, enjoy stunts  
Need active play to let off steam 
Enjoy challenges in balancing  
Like to test limit of own body  
Are hungry at short intervals  
Have insatiable taste for sweets  
Basic skill

/opt/conda/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response: Physical and motor development in young children involves the use of large muscles in activities such as walking, running, jumping, throwing, and lifting. Gross motor skills are related to body awareness, reaction speed, coordination, and strength, while fine motor skills involve activities performed with the fingers in coordination with the eyes such as reaching, grasping, and turning the wrist. These skills help children perform everyday tasks and become independent as they develop. Physical development is a function of maturation, growth, and learning, and opportunities to use gross and fine muscles regularly strengthen physical skills. It is important to remember that physical skills develop from head to toe and from the center of the body outward toward the fingers and toes. Therefore, gross motor skills develop before fine motor skills, which is why children may struggle with handwriting and coloring within lines until they develop more control in their fingers and wris

You:  quit


Goodbye! Have a great day!
